CPFV (Combined Probability and Class Feature Vector)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#marge the column
import pandas as pd

# Define the file paths and prefixes
data_paths = {
    "ACC": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/CPFV (Combined Probability and Class Feature Vector)/CPFV_AAC.csv",
    "CTDC": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/CPFV (Combined Probability and Class Feature Vector)/CPFV_CTDC.csv",
    "CTD": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/CPFV (Combined Probability and Class Feature Vector)/CPFV_CTD.csv",
    "GDC": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/CPFV (Combined Probability and Class Feature Vector)/CPFV_GDC.csv",
    "PAAC": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/CPFV (Combined Probability and Class Feature Vector)/CPFV_PAAC.csv",
    "PCP": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/CPFV (Combined Probability and Class Feature Vector)/CPFV_PCP.csv",
    "TPC": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/CPFV (Combined Probability and Class Feature Vector)/CPFV_TPC.csv",
    "CTDT": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/CPFV (Combined Probability and Class Feature Vector)/CPFV_CTDT.csv",
    "DPC": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/CPFV (Combined Probability and Class Feature Vector)/CPFV_DPC.csv",
    "CTDD": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/CPFV (Combined Probability and Class Feature Vector)/CPFV_CTDD.csv",
}

processed_dfs = []
final_target_column = None  # To store the unique Target column

# Process each file in the dictionary
for prefix, path in data_paths.items():
    # Load the dataset
    df = pd.read_csv(path)

    # If a 'Target' column exists, store it and ensure only one is retained
    if 'Target' in df.columns:
        if final_target_column is None:
            final_target_column = df['Target']  # Retain the first Target column
        df.drop(columns=['Target'], inplace=True)  # Drop from the current dataset

    # Drop the 'True_Label' column if it exists
    df.drop(columns=['True_Label'], inplace=True, errors='ignore')

    # Rename columns with the prefix
    df = df.rename(columns=lambda col: f"{col}_{prefix}" if 'probabilities' not in col else col)

    # Append the processed DataFrame to the list
    processed_dfs.append(df)

# Concatenate all DataFrames column-wise
combined_df = pd.concat(processed_dfs, axis=1)

# Add the retained 'Target' column to the final dataset
if final_target_column is not None:
    combined_df['Target'] = final_target_column

# Save the combined DataFrame
output_file = "/content/Dataset Marge CPFV.csv"
combined_df.to_csv(output_file, index=False)

print(f"Processed dataset saved to: {output_file}")

Processed dataset saved to: /content/Dataset Marge CPFV.csv


In [ ]:
# Check the shape of the merged dataset
print("Shape of the merged dataset:", combined_df.shape)

Shape of the merged dataset: (300, 240)


In [ ]:
df=pd.read_csv("/content/Dataset Marge CPFV.csv")

In [ ]:
df.shape

(300, 240)

In [ ]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 12.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from deap import base, creator, tools, algorithms
import random

# Load the dataset
data_path = '/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/CPFV (Combined Probability and Class Feature Vector)/Optuna_Dataset Marge CPFV.csv'
data = pd.read_csv(data_path)

# Assuming the last column is the target, split features and labels
X = data.iloc[:, :-1]  # Features
y = data.iloc[:, -1]   # Target

# Encode target labels if necessary
if y.dtype == object or np.issubdtype(y.dtype, np.number):  # Handle both string and numeric labels
    le = LabelEncoder()
    y = le.fit_transform(y.astype(str))  # Ensure all targets are treated as strings for classification

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the individual and fitness functions
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(X.columns))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Classifier to evaluate fitness
def evaluate(individual):
    selected_features = [i for i, bit in enumerate(individual) if bit == 1]
    if len(selected_features) == 0:  # Prevent division by zero
        return 0,

    X_train_selected = X_train.iloc[:, selected_features]
    X_val_selected = X_val.iloc[:, selected_features]

    model = RandomForestClassifier(random_state=42)
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_val_selected)

    accuracy = accuracy_score(y_val, y_pred)
    return accuracy,

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Parameters for the Genetic Algorithm
population_size = 50
generations = 20
crossover_probability = 0.8
mutation_probability = 0.1

# Initialize population
population = toolbox.population(n=population_size)

# Run the Genetic Algorithm
result_population, logbook = algorithms.eaSimple(
    population,
    toolbox,
    cxpb=crossover_probability,
    mutpb=mutation_probability,
    ngen=generations,
    verbose=True
)

# Find the best individual
best_individual = tools.selBest(result_population, k=1)[0]
selected_features = [i for i, bit in enumerate(best_individual) if bit == 1 and i < len(X.columns)]  # Bounds check

# Select top 20 features based on their importance
if len(selected_features) > 25:
    feature_importances = pd.Series(best_individual).sort_values(ascending=False)
    selected_features = list(feature_importances.head(25).index)

# Evaluate performance using the top 25 features
X_train_selected = X_train.iloc[:, selected_features]
X_val_selected = X_val.iloc[:, selected_features]

final_model = RandomForestClassifier(random_state=42)
final_model.fit(X_train_selected, y_train)
final_predictions = final_model.predict(X_val_selected)
final_accuracy = accuracy_score(y_val, final_predictions)

print(f"Top 25 Selected features: {selected_features}")
print(f"Final Accuracy with top 25 selected features: {final_accuracy}")

# Save the top 20 selected features
pd.DataFrame({'Selected Features': selected_features}).to_csv('/content/CPFV_top_25_selected_features.csv', index=False)


gen	nevals
0  	50    
1  	40    
2  	37    
3  	33    
4  	39    
5  	40    
6  	38    
7  	45    
8  	29    
9  	46    
10 	42    
11 	45    
12 	42    
13 	44    
14 	37    
15 	38    
16 	44    
17 	40    
18 	40    
19 	36    
20 	41    
Top 25 Selected features: [225, 230, 10, 12, 13, 216, 222, 210, 211, 213, 17, 18, 22, 23, 200, 201, 202, 206, 27, 30, 31, 192, 194, 195, 199]
Final Accuracy with top 25 selected features: 0.9


In [ ]:
selected_feature_columns = data.columns[selected_features]
# Create a filtered dataset with only the selected top 25 features
filtered_data = data[selected_feature_columns.tolist() + [data.columns[-1]]]
filtered_data_path = '/content/CPFV_Top_25_Features.csv'
filtered_data.to_csv(filtered_data_path, index=False)

print(f"Filtered dataset with top 25 features saved to: {filtered_data_path}")

Filtered dataset with top 25 features saved to: /content/CPFV_Top_25_Features.csv


In [ ]:
data=pd.read_csv("/content/CPFV_Top_25_Features.csv")


In [ ]:
data.shape

(300, 26)

Class Feature Vector (CFV)

In [ ]:
#marge the column
import pandas as pd

# Define the file paths and prefixes
data_paths = {
    "ACC": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Class Feature Vector (CFV)/CFV_AAC.csv",
    "CTDC": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Class Feature Vector (CFV)/CFV_CTDC (1).csv",
    "CTD": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Class Feature Vector (CFV)/CFV_CTD (3).csv",
    "GDC": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Class Feature Vector (CFV)/CFV_GDC.csv",
    "PAAC": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Class Feature Vector (CFV)/CFV_PAAC.csv",
    "PCP": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Class Feature Vector (CFV)/CFV_PCP.csv",
    "TPC": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Class Feature Vector (CFV)/CFV_TPC.csv",
    "CTDT": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Class Feature Vector (CFV)/CFV_CTDT.csv",
    "DPC": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Class Feature Vector (CFV)/CFV_DPC (1).csv",
    "CTDD": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Class Feature Vector (CFV)/CFV_CTDD.csv",
}

processed_dfs = []
final_target_column = None  # To store the unique Target column

# Process each file in the dictionary
for prefix, path in data_paths.items():
    # Load the dataset
    df = pd.read_csv(path)

    # If a 'Target' column exists, store it and ensure only one is retained
    if 'Target' in df.columns:
        if final_target_column is None:
            final_target_column = df['Target']  # Retain the first Target column
        df.drop(columns=['Target'], inplace=True)  # Drop from the current dataset

    # Drop the 'True_Label' column if it exists
    df.drop(columns=['True_Label'], inplace=True, errors='ignore')

    # Rename columns with the prefix
    df = df.rename(columns=lambda col: f"{col}_{prefix}" if 'probabilities' not in col else col)

    # Append the processed DataFrame to the list
    processed_dfs.append(df)

# Concatenate all DataFrames column-wise
combined_df = pd.concat(processed_dfs, axis=1)

# Add the retained 'Target' column to the final dataset
if final_target_column is not None:
    combined_df['Target'] = final_target_column

# Save the combined DataFrame
output_file = "/content/Optuna_Dataset Marge CFV.csv"
combined_df.to_csv(output_file, index=False)

print(f"Processed dataset saved to: {output_file}")

Processed dataset saved to: /content/Optuna_Dataset Marge CFV.csv


In [ ]:
# Check the shape of the merged dataset
print("Shape of the merged dataset:", combined_df.shape)

Shape of the merged dataset: (300, 120)


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Class Feature Vector (CFV)/Optuna_Dataset Marge CFV.csv")

In [ ]:
df.shape

(300, 121)

In [ ]:
!pip install deap

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from deap import base, creator, tools, algorithms
import random

# Load the dataset
data_path = '/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Class Feature Vector (CFV)/Optuna_Dataset Marge CFV.csv'
data = pd.read_csv(data_path)

# Assuming the last column is the target, split features and labels
X = data.iloc[:, :-1]  # Features
y = data.iloc[:, -1]   # Target

# Encode target labels if necessary
if y.dtype == object or np.issubdtype(y.dtype, np.number):  # Handle both string and numeric labels
    le = LabelEncoder()
    y = le.fit_transform(y.astype(str))  # Ensure all targets are treated as strings for classification

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the individual and fitness functions
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(X.columns))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Classifier to evaluate fitness
def evaluate(individual):
    selected_features = [i for i, bit in enumerate(individual) if bit == 1]
    if len(selected_features) == 0:  # Prevent division by zero
        return 0,

    X_train_selected = X_train.iloc[:, selected_features]
    X_val_selected = X_val.iloc[:, selected_features]

    model = RandomForestClassifier(random_state=42)
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_val_selected)

    accuracy = accuracy_score(y_val, y_pred)
    return accuracy,

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Parameters for the Genetic Algorithm
population_size = 50
generations = 20
crossover_probability = 0.8
mutation_probability = 0.1

# Initialize population
population = toolbox.population(n=population_size)

# Run the Genetic Algorithm
result_population, logbook = algorithms.eaSimple(
    population,
    toolbox,
    cxpb=crossover_probability,
    mutpb=mutation_probability,
    ngen=generations,
    verbose=True
)

# Find the best individual
best_individual = tools.selBest(result_population, k=1)[0]
selected_features = [i for i, bit in enumerate(best_individual) if bit == 1 and i < len(X.columns)]  # Bounds check

# Select top 20 features based on their importance
if len(selected_features) > 25:
    feature_importances = pd.Series(best_individual).sort_values(ascending=False)
    selected_features = list(feature_importances.head(25).index)

# Evaluate performance using the top 25 features
X_train_selected = X_train.iloc[:, selected_features]
X_val_selected = X_val.iloc[:, selected_features]

final_model = RandomForestClassifier(random_state=42)
final_model.fit(X_train_selected, y_train)
final_predictions = final_model.predict(X_val_selected)
final_accuracy = accuracy_score(y_val, final_predictions)

print(f"Top 25 Selected features: {selected_features}")
print(f"Final Accuracy with top 25 selected features: {final_accuracy}")

# Save the top 20 selected features
pd.DataFrame({'Selected Features': selected_features}).to_csv('/content/CFV_top_25_selected_features.csv', index=False)


/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	50    
1  	34    
2  	36    
3  	44    
4  	37    
5  	38    
6  	38    
7  	41    
8  	44    
9  	40    
10 	40    
11 	37    
12 	38    
13 	35    
14 	46    
15 	38    
16 	35    
17 	36    
18 	30    
19 	45    
20 	42    
Top 25 Selected features: [105, 106, 110, 8, 10, 11, 13, 15, 96, 98, 99, 100, 17, 18, 19, 20, 22, 23, 25, 28, 31, 90, 91, 94, 95]
Final Accuracy with top 25 selected features: 0.9166666666666666


In [ ]:
selected_feature_columns = data.columns[selected_features]
# Create a filtered dataset with only the selected top 20 features
filtered_data = data[selected_feature_columns.tolist() + [data.columns[-1]]]
filtered_data_path = '/content/CFV_Top_25_Features.csv'
filtered_data.to_csv(filtered_data_path, index=False)

print(f"Filtered dataset with top 25 features saved to: {filtered_data_path}")

Filtered dataset with top 25 features saved to: /content/CFV_Top_25_Features.csv


PFV (Probability Feature Vector)

In [ ]:
#marge the column
import pandas as pd

# Define the file paths and prefixes
data_paths = {
    "ACC": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Probabilites Dataset Marge/N_AAC_OPTUNA_probability_predictions.csv",
    "CTDC": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Probabilites Dataset Marge/N_CTDC_OPTUNA_probability_predictions.csv",
    "CTD": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Probabilites Dataset Marge/N_CTD_OPTUNA_probability_predictions.csv",
    "GDC": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Probabilites Dataset Marge/N_GDC_OPTUNA_probability_predictions.csv",
    "PAAC": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Probabilites Dataset Marge/N_PAAC_OPTUNA_probability_predictions.csv",
    "PCP": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Probabilites Dataset Marge/N_PCP_OPTUNA_probability_predictions.csv",
    "TPC": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Probabilites Dataset Marge/N_TPC_OPTUNA_probability_predictions.csv",
    "CTDT": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Probabilites Dataset Marge/N_CTDT_OPTUNA_probability_predictions.csv",
    "DPC": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Probabilites Dataset Marge/N_DPC_OPTUNA_probability_predictions.csv",
    "CTDD": "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Probabilites Dataset Marge/N_CTDD_OPTUNA_probability_predictions.csv",
}

processed_dfs = []
final_target_column = None  # To store the unique Target column

# Process each file in the dictionary
for prefix, path in data_paths.items():
    # Load the dataset
    df = pd.read_csv(path)

    # If a 'Target' column exists, store it and ensure only one is retained
    if 'Target' in df.columns:
        if final_target_column is None:
            final_target_column = df['Target']  # Retain the first Target column
        df.drop(columns=['Target'], inplace=True)  # Drop from the current dataset

    # Drop the 'True_Label' column if it exists
    df.drop(columns=['True_Label'], inplace=True, errors='ignore')

    # Rename columns with the prefix
    df = df.rename(columns=lambda col: f"{col}_{prefix}" if 'probabilities' not in col else col)

    # Append the processed DataFrame to the list
    processed_dfs.append(df)

# Concatenate all DataFrames column-wise
combined_df = pd.concat(processed_dfs, axis=1)

# Add the retained 'Target' column to the final dataset
if final_target_column is not None:
    combined_df['Target'] = final_target_column

# Save the combined DataFrame
output_file = "/content/Optuna_Dataset Marge PFV.csv"
combined_df.to_csv(output_file, index=False)

print(f"Processed dataset saved to: {output_file}")

Processed dataset saved to: /content/Optuna_Dataset Marge PFV.csv


In [ ]:
# Check the shape of the merged dataset
print("Shape of the merged dataset:", combined_df.shape)

Shape of the merged dataset: (300, 121)


In [ ]:
df=pd.read_csv("/content/Optuna_Dataset Marge PFV.csv")

In [ ]:
df.shape

(300, 121)

In [ ]:
df.columns

Index(['SVM_ACC', 'Decision Tree_ACC', 'Random Forest_ACC',
       'Logistic Regression_ACC', 'k-NN_ACC', 'Naive Bayes_ACC',
       'Gradient Boosting_ACC', 'XGBoost_ACC', 'LightGBM_ACC', 'AdaBoost_ACC',
       ...
       'Logistic Regression_CTDD', 'k-NN_CTDD', 'Naive Bayes_CTDD',
       'Gradient Boosting_CTDD', 'XGBoost_CTDD', 'LightGBM_CTDD',
       'AdaBoost_CTDD', 'Neural Network_CTDD', 'MLP_CTDD', 'Target'],
      dtype='object', length=121)

In [ ]:
!pip install deap

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from deap import base, creator, tools, algorithms
import random

# Load the dataset
data_path = '/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/Probabilites Dataset Marge/Optuna_Dataset Marge PFV (1).csv'
data = pd.read_csv(data_path)

# Assuming the last column is the target, split features and labels
X = data.iloc[:, :-1]  # Features
y = data.iloc[:, -1]   # Target

# Encode target labels if necessary
if y.dtype == object or np.issubdtype(y.dtype, np.number):  # Handle both string and numeric labels
    le = LabelEncoder()
    y = le.fit_transform(y.astype(str))  # Ensure all targets are treated as strings for classification

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the individual and fitness functions
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(X.columns))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Classifier to evaluate fitness
def evaluate(individual):
    selected_features = [i for i, bit in enumerate(individual) if bit == 1]
    if len(selected_features) == 0:  # Prevent division by zero
        return 0,

    X_train_selected = X_train.iloc[:, selected_features]
    X_val_selected = X_val.iloc[:, selected_features]

    model = RandomForestClassifier(random_state=42)
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_val_selected)

    accuracy = accuracy_score(y_val, y_pred)
    return accuracy,

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Parameters for the Genetic Algorithm
population_size = 50
generations = 20
crossover_probability = 0.8
mutation_probability = 0.1

# Initialize population
population = toolbox.population(n=population_size)

# Run the Genetic Algorithm
result_population, logbook = algorithms.eaSimple(
    population,
    toolbox,
    cxpb=crossover_probability,
    mutpb=mutation_probability,
    ngen=generations,
    verbose=True
)

# Find the best individual
best_individual = tools.selBest(result_population, k=1)[0]
selected_features = [i for i, bit in enumerate(best_individual) if bit == 1 and i < len(X.columns)]  # Bounds check

# Select top 20 features based on their importance
if len(selected_features) > 25:
    feature_importances = pd.Series(best_individual).sort_values(ascending=False)
    selected_features = list(feature_importances.head(25).index)

# Evaluate performance using the top 25 features
X_train_selected = X_train.iloc[:, selected_features]
X_val_selected = X_val.iloc[:, selected_features]

final_model = RandomForestClassifier(random_state=42)
final_model.fit(X_train_selected, y_train)
final_predictions = final_model.predict(X_val_selected)
final_accuracy = accuracy_score(y_val, final_predictions)

print(f"Top 25 Selected features: {selected_features}")
print(f"Final Accuracy with top 25 selected features: {final_accuracy}")

# Save the top 20 selected features
pd.DataFrame({'Selected Features': selected_features}).to_csv('/content/PFV_top_25_selected_features.csv', index=False)


/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	50    
1  	40    
2  	44    
3  	41    
4  	46    
5  	40    
6  	41    
7  	39    
8  	29    
9  	39    
10 	43    
11 	33    
12 	42    
13 	40    
14 	40    
15 	46    
16 	38    
17 	41    
18 	43    
19 	37    
20 	42    
Top 25 Selected features: [107, 109, 111, 9, 11, 12, 96, 97, 98, 101, 16, 18, 20, 21, 22, 23, 88, 90, 93, 95, 24, 25, 27, 31, 80]
Final Accuracy with top 25 selected features: 0.9


In [ ]:
selected_feature_columns = data.columns[selected_features]
# Create a filtered dataset with only the selected top 25 features
filtered_data = data[selected_feature_columns.tolist() + [data.columns[-1]]]
filtered_data_path = '/content/PFV_Top_25_Features.csv'
filtered_data.to_csv(filtered_data_path, index=False)

print(f"Filtered dataset with top 20 features saved to: {filtered_data_path}")

Filtered dataset with top 20 features saved to: /content/PFV_Top_25_Features.csv


Three technique dataset are marge

In [ ]:
#marge the column
import pandas as pd

# Define the file paths and prefixes
data_paths = {
    "CFV": "/content/CFV_Top_25_Features.csv",
    "CPFV": "/content/CPFV_Top_25_Features.csv",
    "PFV": "/content/PFV_Top_25_Features.csv",
}

processed_dfs = []
final_target_column = None  # To store the unique Target column

# Process each file in the dictionary
for prefix, path in data_paths.items():
    # Load the dataset
    df = pd.read_csv(path)

    # If a 'Target' column exists, store it and ensure only one is retained
    if 'Target' in df.columns:
        if final_target_column is None:
            final_target_column = df['Target']  # Retain the first Target column
        df.drop(columns=['Target'], inplace=True)  # Drop from the current dataset

    # Drop the 'True_Label' column if it exists
    df.drop(columns=['True_Label'], inplace=True, errors='ignore')

    # Rename columns with the prefix
    df = df.rename(columns=lambda col: f"{col}_{prefix}" if 'probabilities' not in col else col)

    # Append the processed DataFrame to the list
    processed_dfs.append(df)

# Concatenate all DataFrames column-wise
combined_df = pd.concat(processed_dfs, axis=1)

# Add the retained 'Target' column to the final dataset
if final_target_column is not None:
    combined_df['Target'] = final_target_column

# Save the combined DataFrame
output_file = "/content/three_T_Marge.csv"
combined_df.to_csv(output_file, index=False)

print(f"Processed dataset saved to: {output_file}")

Processed dataset saved to: /content/three_T_Marge.csv


In [ ]:
df=pd.read_csv("/content/three_T_Marge.csv")

In [ ]:
df.shape

(300, 76)

Deep learning approach combining Conv1D, LSTM, and Dense layers

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# Load the combined AAC dataset
dataset_path = "/content/three_T_Marge.csv"
data = pd.read_csv(dataset_path)

# Ensure the dataset contains the 'Target' column
if 'Target' not in data.columns:
    raise ValueError("The dataset must include a 'Target' column for binary classification.")

# Separate features and labels
X = data.drop(columns=['Target']).values  # Features (all columns except 'Target')
y = data['Target'].values                 # Labels (the 'Target' column)

# Split data into training and validation sets (80-20 split)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Reshape input for Conv1D
X_train = X_train[..., np.newaxis]  # Adding channel dimension for Conv1D input
X_val = X_val[..., np.newaxis]      # Adding channel dimension for Conv1D input

# Model Architecture
model = Sequential()

# Stacked Conv1D layers with BatchNormalization and Dropout
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv1D(filters=128, kernel_size=5, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'))
model.add(BatchNormalization())
# Removed problematic pooling layer here to avoid negative dimension issue
model.add(Dropout(0.3))

# LSTM layer for sequential dependencies
model.add(LSTM(64, return_sequences=False, activation='relu'))

# Dense Layers for final prediction with Dropout for regularization
model.add(Dense(128, activation='swish'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='swish'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))   # Output layer for binary classification

# Compile the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model Summary
model.summary()

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32, verbose=1)

# Evaluate the model
val_predictions = (model.predict(X_val) > 0.5).astype(int)
accuracy = accuracy_score(y_val, val_predictions)

print("\nValidation Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_val, val_predictions))

# Save the trained model
model.save("mmCombinedDataset_AAC_model.h5")
print("Model saved as 'mmCombinedDataset_AAC_model.h5'.")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)                    │ (None, 75, 64)              │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 75, 64)              │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 37, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 37, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_4 (Conv1D)                    │ (None, 37, 128)             │          41,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 37, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_3 (MaxPooling1D)       │ (None, 18, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 18, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_5 (Conv1D)                    │ (None, 18, 256)             │          98,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 18, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 18, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          82,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 240,513 (939.50 KB)

 Trainable params: 239,617 (936.00 KB)

 Non-trainable params: 896 (3.50 KB)

Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.7142 - loss: 0.5364 - val_accuracy: 0.9444 - val_loss: 0.6480
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.9209 - loss: 0.2069 - val_accuracy: 0.9111 - val_loss: 0.6281
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9110 - loss: 0.2221 - val_accuracy: 0.9111 - val_loss: 0.6379
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9226 - loss: 0.1864 - val_accuracy: 0.9111 - val_loss: 0.6098
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9316 - loss: 0.1738 - val_accuracy: 0.7889 - val_loss: 0.6011
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9039 - loss: 0.1624 - val_accuracy: 0.8556 - val_loss: 0.5690
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9571 - loss: 0.1387 - val_accuracy: 0.7778 - val_loss: 0.5354
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9352 - loss: 0.1760 - val_accuracy: 0.8667 - val_loss: 


Validation Accuracy: 0.9222222222222223

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.91      0.92        45
           1       0.91      0.93      0.92        45

    accuracy                           0.92        90
   macro avg       0.92      0.92      0.92        90
weighted avg       0.92      0.92      0.92        90

Model saved as 'mmCombinedDataset_AAC_model.h5'.


PeptidePredictor_CNN

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv1D, BatchNormalization, Dropout, GlobalMaxPooling1D, Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.metrics import classification_report
from keras import regularizers

# Load Dataset
file_path = '/content/three_T_Marge.csv'
data = pd.read_csv(file_path)

# Separate Features and Labels
X = data.iloc[:, :-1].values  # Features
y = data.iloc[:, -1].values   # Labels

# Handle missing values (imputation)
imputer = SimpleImputer(strategy='mean')  # You can also use median or most_frequent
X = imputer.fit_transform(X)

# Normalize Features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# SMOTE for oversampling the minority class
smote = SMOTE(sampling_strategy='auto', random_state=42)
X, y = smote.fit_resample(X, y)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Build Model (same as the one discussed previously)
def build_model(input_dim):
    inputs = Input(shape=(input_dim, 1))  # 1D CNN expects a 3D input shape (samples, features, 1)

    # Conv1D Layer
    x = Conv1D(64, kernel_size=3, activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    # Another Conv1D Layer
    x = Conv1D(128, kernel_size=3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    # Max Pooling
    x = GlobalMaxPooling1D()(x)

    # Dense Layer
    x = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = Dropout(0.4)(x)

    # Output Layer
    outputs = Dense(1, activation='sigmoid')(x)  # For binary classification

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Reshaping input for Conv1D (samples, features, 1)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Get Input Dimension
input_dim = X_train.shape[1]

# Initialize Model
model = build_model(input_dim)

# Early Stopping (optional, but can be added for faster convergence)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the Model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.4f}")

# Evaluate the Model using Classification Report
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))

# Save the Model
model.save('/content/improved_probability_based_model.h5')


Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 804ms/step - accuracy: 0.7146 - loss: 2.4095 - val_accuracy: 0.9000 - val_loss: 1.8010
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9570 - loss: 1.6022 - val_accuracy: 0.9000 - val_loss: 1.7221
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9148 - loss: 1.6979 - val_accuracy: 0.8889 - val_loss: 1.6829
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9565 - loss: 1.3416 - val_accuracy: 0.9000 - val_loss: 1.6509
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9585 - loss: 1.3087 - val_accuracy: 0.9000 - val_loss: 1.6292
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9099 - loss: 1.3444 - val_accuracy: 0.8889 - val_loss: 1.6067
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9174 - loss: 1.2737 - val_accuracy: 0.8778 - val_loss: 1.5738
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9191 - loss: 1.2795 - val_accuracy: 0.8778 - val_los

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 390ms/step

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 174ms/step


              precision    recall  f1-score   support

           0       0.88      0.83      0.85        42
           1       0.86      0.90      0.88        48

    accuracy                           0.87        90
   macro avg       0.87      0.86      0.87        90
weighted avg       0.87      0.87      0.87        90



In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.3 MB/s eta 0:00:00


In [ ]:
import optuna
from keras.models import Model
from keras.layers import Input, Conv1D, BatchNormalization, Dropout, GlobalMaxPooling1D, Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn.metrics import classification_report

# Load the dataset
file_path = '/content/three_T_Marge.csv'
data = pd.read_csv(file_path)

# Separate features and labels
X = data.iloc[:, :-1].values  # Features
y = data.iloc[:, -1].values   # Labels

# Handle missing values (imputation)
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# SMOTE for oversampling the minority class
smote = SMOTE(sampling_strategy='auto', random_state=42)
X, y = smote.fit_resample(X, y)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Reshaping the input for Conv1D (samples, features, 1)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Objective function for Optuna
def objective(trial):
    # Hyperparameters to tune
    filters_1 = trial.suggest_int('filters_1', 32, 128)
    filters_2 = trial.suggest_int('filters_2', 64, 256)
    kernel_size_1 = trial.suggest_int('kernel_size_1', 3, 5)
    kernel_size_2 = trial.suggest_int('kernel_size_2', 3, 5)
    dropout_1 = trial.suggest_float('dropout_1', 0.2, 0.5)
    dropout_2 = trial.suggest_float('dropout_2', 0.2, 0.5)
    dense_units = trial.suggest_int('dense_units', 64, 256)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)

    # Build model
    inputs = Input(shape=(X_train.shape[1], 1))  # 1D CNN expects a 3D input shape (samples, features, 1)

    # First Conv1D Layer
    x = Conv1D(filters_1, kernel_size=kernel_size_1, activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(dropout_1)(x)

    # Second Conv1D Layer
    x = Conv1D(filters_2, kernel_size=kernel_size_2, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_2)(x)

    # Max Pooling
    x = GlobalMaxPooling1D()(x)

    # Dense Layer
    x = Dense(dense_units, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = Dropout(0.4)(x)

    # Output Layer
    outputs = Dense(1, activation='sigmoid')(x)  # For binary classification

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=50,
        batch_size=32,
        callbacks=[early_stopping],
        verbose=0
    )

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy  # We want to maximize accuracy

# Create an Optuna study and start optimizing
study = optuna.create_study(direction='maximize')  # We want to maximize the accuracy

# Optimize the hyperparameters
study.optimize(objective, n_trials=50)  # n_trials is the number of different hyperparameter sets to try

# Print the best hyperparameters and accuracy
print(f"Best Hyperparameters: {study.best_params}")
print(f"Best Accuracy: {study.best_value:.4f}")


[I 2025-01-17 05:30:59,232] A new study created in memory with name: no-name-02d373dd-7bad-4475-a4b9-e561f06fa2f9
<ipython-input-27-c5e6fc210871>:51: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
[I 2025-01-17 05:31:27,946] Trial 0 finished with value: 0.9111111164093018 and parameters: {'filters_1': 57, 'filters_2': 68, 'kernel_size_1': 5, 'kernel_size_2': 4, 'dropout_1': 0.48103160164322223, 'dropout_2': 0.3513107797735918, 'dense_units': 84, 'learning_rate': 1.1751416211179787e-05}. Best is trial 0 with value: 0.9111111164093018.
[I 2025-01-17 05:31:45,921] Trial 1 finished with value: 0.8777777552604675 and parameters: {'filters_1': 34, 'filters_2': 199, 'kernel_size_1': 5, 'kernel_size_2': 3, 'dropout_1': 0.3342055028708109, 'dropout_2': 0.35325868372759

Best Hyperparameters: {'filters_1': 57, 'filters_2': 68, 'kernel_size_1': 5, 'kernel_size_2': 4, 'dropout_1': 0.48103160164322223, 'dropout_2': 0.3513107797735918, 'dense_units': 84, 'learning_rate': 1.1751416211179787e-05}
Best Accuracy: 0.9111
